In [1]:
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import os
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, train_mlp, test_mlp, train_mlpbeta, test_mlpbeta, train_cnn, test_cnn
import session_new as sess

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch
set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'AB'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,feat,feat_sq = pickle.load(f)

In [3]:
d = {'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'manual','scaler_load':False,'feat_type':'feat'}
sub = 2

d_in = sess.Sess(**d)

ind = (params[:,0] == sub) & (params[:,3] == d_in.train_grp)
if np.sum(ind):
    trainmlp, validmlp, traincnn, validcnn, y_train_clean, y_valid_clean, x_train_noise_mlp, x_train_noise_cnn, emg_scale, scaler = prd.prep_train_data(d_in,raw,params,sub)

Loading training data: traindata_manual/AB2_traindata_2.p


In [3]:
## Training 
sub = 2
train_grp = 2
n_train = 'fullallmix4'
train_scale = 5
cv_type = 'manual'
scaler_load = False
feat_type = 'feat'

In [4]:
ind = (params[:,0] == sub) & (params[:,3] == train_grp)

x_train, x_test, x_valid, p_train, p_test, p_valid = prd.train_data_split(raw,params,sub,sub_type,dt=cv_type,load=True,train_grp=train_grp)

emg_scale = np.ones((np.size(x_train,1),1))
for i in range(np.size(x_train,1)):
    emg_scale[i] = 5/np.max(np.abs(x_train[:,i,:]))
x_train = x_train*emg_scale
x_valid = x_valid*emg_scale

x_train_noise, x_train_clean, y_train_clean = prd.add_noise(x_train, p_train, sub, n_train, train_scale)
x_valid_noise, x_valid_clean, y_valid_clean = prd.add_noise(x_valid, p_valid, sub, n_train, train_scale)

# shuffle data to make even batches
x_train_noise, x_train_clean, y_train_clean = shuffle(x_train_noise, x_train_clean, y_train_clean, random_state = 0)

# Extract features
scaler = MinMaxScaler(feature_range=(0,1))
x_train_noise_cnn, scaler = prd.extract_scale(x_train_noise,scaler,scaler_load,ft=feat_type,emg_scale=emg_scale) 
x_valid_noise_cnn, _ = prd.extract_scale(x_valid_noise,scaler,ft=feat_type,emg_scale=emg_scale)
x_train_noise_cnn = x_train_noise_cnn.astype('float32')
x_valid_noise_cnn = x_valid_noise_cnn.astype('float32')

# reshape data for nonconvolutional network
x_train_noise_mlp = x_train_noise_cnn.reshape(x_train_noise_cnn.shape[0],-1)
x_valid_noise_mlp = x_valid_noise_cnn.reshape(x_valid_noise_cnn.shape[0],-1)

# create batches
trainmlp = tf.data.Dataset.from_tensor_slices((x_train_noise_mlp, y_train_clean)).batch(128)
validmlp = tf.data.Dataset.from_tensor_slices((x_valid_noise_mlp, y_valid_clean)).batch(128)
traincnn = tf.data.Dataset.from_tensor_slices((x_train_noise_cnn, y_train_clean)).batch(128)
validcnn = tf.data.Dataset.from_tensor_slices((x_valid_noise_cnn, y_valid_clean)).batch(128)

Loading training data: traindata_manual/AB2_traindata_2.p


In [5]:
mlp = MLP()
mlp_beta = MLPbeta()
cnn = CNN()

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [6]:
EPOCHS = 10
models = ['mlp','mlpbeta','cnn']

for model in models:
    print('Training ' + model)
    for epoch in range(EPOCHS):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        # Train MLP
        if model == 'mlp':
            for x, y in trainmlp:
                train_mlp(x, y, mlp, loss_fn, optimizer, train_loss, train_accuracy)
            for x_test, y_test in validmlp:
                test_mlp(x_test, y_test, mlp, loss_fn, test_loss, test_accuracy)
        # Train MLP Beta
        elif model == 'mlpbeta':
            for x, y in trainmlp:
                train_mlpbeta(x, y, mlp_beta, loss_fn, optimizer, train_loss, train_accuracy)
            for x_test, y_test in validmlp:
                test_mlpbeta(x_test, y_test, mlp_beta, loss_fn, test_loss, test_accuracy)
        # Train CNN
        elif model == 'cnn':
            for x, y in traincnn:
                train_cnn(x, y, cnn, loss_fn, optimizer, train_loss, train_accuracy)
            for x_test, y_test in validcnn:
                test_cnn(x_test, y_test, cnn, loss_fn, test_loss, test_accuracy)

        print(
            f'Epoch {epoch + 1}, '
            f'Loss: {train_loss.result()}, '
            f'Accuracy: {train_accuracy.result() * 100}, '
            f'Test Loss: {test_loss.result()}, '
            f'Test Accuracy: {test_accuracy.result() * 100}'
        )

Training mlp
Epoch 1, Loss: 1.9409295320510864, Accuracy: 16.146825790405273, Test Loss: 1.920284390449524, Test Accuracy: 20.428571701049805
Epoch 2, Loss: 1.8784139156341553, Accuracy: 26.821428298950195, Test Loss: 1.8290220499038696, Test Accuracy: 30.571428298950195
Epoch 3, Loss: 1.8007692098617554, Accuracy: 35.36111068725586, Test Loss: 1.7818107604980469, Test Accuracy: 36.476192474365234
Epoch 4, Loss: 1.7651963233947754, Accuracy: 39.603172302246094, Test Loss: 1.7600924968719482, Test Accuracy: 40.238094329833984
Epoch 5, Loss: 1.7422057390213013, Accuracy: 42.15079116821289, Test Loss: 1.7425862550735474, Test Accuracy: 41.36507797241211
Epoch 6, Loss: 1.7251882553100586, Accuracy: 43.873016357421875, Test Loss: 1.722000002861023, Test Accuracy: 45.28571319580078
Epoch 7, Loss: 1.70674467086792, Accuracy: 46.32142639160156, Test Loss: 1.7046308517456055, Test Accuracy: 46.42856979370117
Epoch 8, Loss: 1.6951076984405518, Accuracy: 47.404762268066406, Test Loss: 1.694179892

In [7]:
mlp_enc = mlp.get_layer(name='enc')
mlpbeta_enc = mlp_beta.get_layer(name='enc')
cnn_enc = cnn.get_layer(name='enc')

mlp_aligned = mlp_enc(x_train_noise_mlp).numpy()
mlp_beta_aligned = mlpbeta_enc(x_train_noise_mlp).numpy()
cnn_aligned = cnn_enc(x_train_noise_cnn).numpy()

y_train_aligned = np.argmax(y_train_clean, axis=1)[...,np.newaxis]
w_mlp, c_mlp,_, _, _ = train_lda(mlp_aligned,y_train_aligned)
w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlp_beta_aligned,y_train_aligned)
w_cnn, c_cnn,_, _, _ = train_lda(cnn_aligned,y_train_aligned)

In [8]:
test_grp = 4
cv_type = 'manual'
n_test = 'partposrealmixeven24'

with open('real_noise/all_real_noise.p', 'rb') as f:
    real_noise_temp, _ = pickle.load(f)

_, x_test, _, _, p_test, _ = prd.train_data_split(raw,params,sub,sub_type,dt=cv_type,train_grp=test_grp)
clean_size = int(np.size(x_test,axis=0))
x_test = x_test*emg_scale

x_test_noise, x_test_clean, y_test_clean = prd.add_noise(x_test, p_test, sub, n_test, 1, real_noise=real_noise_temp, emg_scale = emg_scale)
x_test_cnn, _ = prd.extract_scale(x_test_noise,scaler,ft=feat_type,emg_scale=emg_scale)
x_test_cnn = x_test_cnn.astype('float32')
x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)

mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
y_test_aligned = np.argmax(y_test_clean, axis=1)[...,np.newaxis]

clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test_clean,mlp,clean_size)
clean_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
noisy_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'MLP ---- '
    f'Clean Accuracy: {clean_acc * 100:.2f}, '
    f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
    f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
    f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
)

clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test_clean,mlp_beta,clean_size)
clean_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
noisy_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'MLPB ---- '
    f'Clean Accuracy: {clean_acc * 100:.2f}, '
    f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
    f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
    f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
)

clean_acc, noisy_acc = eval_nn(x_test_cnn, y_test_clean,cnn,clean_size)
clean_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
noisy_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
print(
    f'CNN ---- '
    f'Clean Accuracy: {clean_acc * 100:.2f}, '
    f'Noisy Accuracy: {noisy_acc * 100:.2f}, '
    f'LDA Clean Accuracy: {clean_lda * 100:.2f}, '
    f'LDA Noisy Accuracy: {noisy_lda * 100:.2f}'
)

Loading training data: traindata_manual/AB2_traindata_4.p
MLP ---- Clean Accuracy: 74.23, Noisy Accuracy: 64.86, LDA Clean Accuracy: 69.86, LDA Noisy Accuracy: 65.14
MLPB ---- Clean Accuracy: 76.57, Noisy Accuracy: 64.20, LDA Clean Accuracy: 77.17, LDA Noisy Accuracy: 60.20
CNN ---- Clean Accuracy: 85.69, Noisy Accuracy: 74.31, LDA Clean Accuracy: 85.40, LDA Noisy Accuracy: 71.86


In [ ]:
MLP ---- Clean Accuracy: 74.86, Noisy Accuracy: 64.49, LDA Clean Accuracy: 49.63, LDA Noisy Accuracy: 50.80
MLPB ---- Clean Accuracy: 76.83, Noisy Accuracy: 65.97, LDA Clean Accuracy: 80.29, LDA Noisy Accuracy: 67.94
CNN ---- Clean Accuracy: 85.37, Noisy Accuracy: 72.77, LDA Clean Accuracy: 82.91, LDA Noisy Accuracy: 68.17

